<a href="https://colab.research.google.com/github/MatteoZanella/NLU-assignement-1/blob/main/NLU_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLU assignment n.1

## Part A: Working with Dependency Graphs
The objective of the assignment is to learn how to work with dependency graphs by defining functions.

Read [spaCy documentation on dependency parser](https://spacy.io/api/dependencyparser) to learn provided methods.

In [ ]:
# Imports
import spacy
from spacy import displacy

# HELPER FUNCTIONS
# get the doc from a sentence
def doc_of(sentence: str) -> spacy.tokens.Doc:
  nlp = spacy.load("en_core_web_sm")
  return nlp(sentence)

In [ ]:
# Example sentence (for testing)
example = "Credit and mortgage account holders must submit their requests."
# Example sentence visualization
displacy.render(doc_of(example), style='dep', jupyter=True)

### Task A1
Extract a path of dependency relations from the ROOT to a token

The best way to match is from below

In [ ]:
from collections import deque


def dependency_path(sentence: str):
  doc = doc_of(sentence)
  root = doc[:].root
  paths = {root: []}
  tokens = deque([root])
  # BFS search from the root
  while tokens:
    token = tokens.popleft()
    paths[token] = [*paths[token.head], token.dep_]
    tokens.extend(token.children)
  return paths

#### Testing A1

In [ ]:
# Task A1 testing
paths = dependency_path(example)
for token in paths:
  print(f"{token}: {paths[token]}")

submit: ['ROOT']
holders: ['ROOT', 'nsubj']
must: ['ROOT', 'aux']
requests: ['ROOT', 'dobj']
.: ['ROOT', 'punct']
account: ['ROOT', 'nsubj', 'compound']
their: ['ROOT', 'dobj', 'poss']
Credit: ['ROOT', 'nsubj', 'compound', 'nmod']
and: ['ROOT', 'nsubj', 'compound', 'nmod', 'cc']
mortgage: ['ROOT', 'nsubj', 'compound', 'nmod', 'conj']


### Task A2
Extract subtree of a dependents given a token

In [ ]:
def dependents_tree(sentence: str):
  doc = doc_of(sentence)
  return {token: [*token.subtree] for token in doc}

#### Testing A2

In [ ]:
# Task A2 testing
paths = dependents_tree(example)
for token in paths:
  print(f"{token}: {paths[token]}")

Credit: [Credit, and, mortgage]
and: [and]
mortgage: [mortgage]
account: [Credit, and, mortgage, account]
holders: [Credit, and, mortgage, account, holders]
must: [must]
submit: [Credit, and, mortgage, account, holders, must, submit, their, requests, .]
their: [their]
requests: [their, requests]
.: [.]


### Task A3
Check if a given list of tokens (ordered list of words from the sentence) forms a subtree


In [ ]:
def is_dependents_tree(sentence: str, sequence: [str]):
  doc = doc_of(sentence)
  sequence_set = set(sequence)
  # We use the first word as anchor to find the within-sequence root. With 
  # repetitions of the anchor word, at least one should have a within-sequence
  # root that is the root of the sequence
  anchor = sequence[0]
  for token in doc:
    if token.text == anchor:
      # Find the within-sequence root
      root = token
      while root != root.head and root.head.text in sequence_set:
        root = root.head
      # Check if the within-sequence root is the sequence root
      if sequence == [token.text for token in root.subtree]:
        return True
  return False


#### Testing A3

In [ ]:
# Task A3 testing
trees = [["Credit", "and", "mortgage"],
         ["and", "Credit", "mortgage"],
         ["must", "submit", "their"]]
for tree in trees:
  print(f"{tree}: {is_dependents_tree(example, tree)}")

['Credit', 'and', 'mortgage']: True
['and', 'Credit', 'mortgage']: False
['must', 'submit', 'their']: False


### Task A4
 Identify the head of a span, given its tokens: find the head/root of a phrase

In [ ]:
def head_of(sentence: str):
  doc = doc_of(sentence)
  return doc[:].root

#### Testing A4

In [ ]:
# Task A4 testing
examples = [example, "man with a telescope", "has the tower tripped down?"]
for ex in examples:
  print(f"{ex}: {head_of(ex)}")

Credit and mortgage account holders must submit their requests.: submit
man with a telescope: man
has the tower tripped down?: tripped


### Task A5
Extract sentence subject, direct object and indirect object spans. Each span lenght is 1, for the single word.

`iobj` is [not a parsed dependency](https://spacy.io/models/en): `dative` is parsed instead.

In [ ]:
def interesting_spans(sentence: str):
  doc = doc_of(sentence)
  spans = {'nsubj': [], 'dobj': [], 'dative':[]}
  for token in doc:
    if token.dep_ == 'nsubj' or token.dep_ == 'dobj' or token.dep_ == 'dative':
      spans[token.dep_].append(doc[token.i:token.i+1])
  return spans

#### Testing A5

In [ ]:
print(interesting_spans(example))
print(interesting_spans("I read her the letter"))
print(interesting_spans("They normally give refugees shelter."))

{'nsubj': [holders], 'dobj': [requests], 'dative': []}
{'nsubj': [I], 'dobj': [letter], 'dative': [her]}
{'nsubj': [They], 'dobj': [shelter], 'dative': [refugees]}


## Part B: Training Transition-Based Dependency Parser
This part is optional and advanced

### Task B1
Modify [NLTK Transition parser](https://github.com/nltk/nltk/blob/develop/nltk/parse/transitionparser.py)'s `Configuration` class to use better features.

### Task B2
Evaluate the features, comparing the performance to the original.

### Task B3
Replace `SVM` classifier with an alternative of your choice.